In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from datasets import Dataset, ClassLabel, Features, Value
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import evaluate

# -----------------------------
# Config
# -----------------------------
STUDENT_MODEL_NAME = "distilbert-base-uncased"
TEACHER_MODEL_NAME = "bert-base-uncased"
NUM_LABELS = 2
MAX_LENGTH = 256
SEED = 12

# KD hyperparams
TEMPERATURE = 2.0
ALPHA = 0.5

# -----------------------------
# Data (Restaurant reviews.csv)
# -----------------------------
CSV_PATH = "Restaurant_reviews.csv"

df = pd.read_csv(CSV_PATH)

# Drop junk columns if present
for col in list(df.columns):
    if col.startswith("Unnamed") or col.isdigit():
        df = df.drop(columns=[col])

# Expect columns: Review (text) and Rating (1–5)
df = df.rename(columns={"Review": "text"})
df = df.dropna(subset=["text", "Rating"])

df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df = df.dropna(subset=["Rating"])

# Binary sentiment:
#   1–2 → negative (0)
#   4–5 → positive (1)
#   3 → drop (neutral)
df = df[df["Rating"] != 3].copy()
df["labels"] = (df["Rating"] >= 4).astype(int)

# -----------------------------
# HuggingFace Dataset + ClassLabel
# -----------------------------
features = Features(
    {
        "text": Value("string"),
        "labels": ClassLabel(names=["negative", "positive"]),
    }
)

ds = Dataset.from_pandas(
    df[["text", "labels"]],
    preserve_index=False,
).cast(features)

splits = ds.train_test_split(
    test_size=0.2,
    seed=SEED,
    stratify_by_column="labels",
)

dataset = {
    "train": splits["train"],
    "validation": splits["test"],
}

# -----------------------------
# Tokenization
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LENGTH,
    )

tokenized = {
    "train": dataset["train"].map(tokenize_fn, batched=True),
    "validation": dataset["validation"].map(tokenize_fn, batched=True),
}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Keep only model inputs
keep_cols = ["input_ids", "attention_mask", "labels"]
for split in ["train", "validation"]:
    tokenized[split] = tokenized[split].remove_columns(
        [c for c in tokenized[split].column_names if c not in keep_cols]
    )

# -----------------------------
# Metrics
# -----------------------------
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(
            predictions=preds,
            references=labels,
        )["accuracy"]
    }

# -----------------------------
# 1) Train Teacher
# -----------------------------
teacher = AutoModelForSequenceClassification.from_pretrained(
    TEACHER_MODEL_NAME,
    num_labels=NUM_LABELS,
)

teacher_args = TrainingArguments(
    output_dir="teacher-bert-restaurants",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=200,
    seed=SEED,
    report_to="none",
    save_strategy="no",
)

teacher_trainer = Trainer(
    model=teacher,
    args=teacher_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

teacher_trainer.train()
print("Teacher eval:", teacher_trainer.evaluate())

# Freeze teacher for KD
teacher_trainer.model.eval()
for p in teacher_trainer.model.parameters():
    p.requires_grad = False

# -----------------------------
# 2) Train Student with KD
# -----------------------------
student = AutoModelForSequenceClassification.from_pretrained(
    STUDENT_MODEL_NAME,
    num_labels=NUM_LABELS,
)

class KDTrainer(Trainer):
    """
    L = alpha * CE(y, student)
      + (1 - alpha) * T^2 * KL(teacher || student)
    """
    def __init__(self, teacher_model, temperature=2.0, alpha=0.5, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.temperature = temperature
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None, **kwargs):
        labels = inputs.get("labels")

        # Student forward
        student_outputs = model(**inputs)
        student_logits = student_outputs.logits

        # CE loss (compute ourselves to be robust)
        ce_loss = F.cross_entropy(student_logits, labels)

        # Teacher forward (no grad)
        with torch.no_grad():
            teacher_outputs = self.teacher_model(**inputs)
            teacher_logits = teacher_outputs.logits

        T = self.temperature
        student_log_probs = F.log_softmax(student_logits / T, dim=-1)
        teacher_probs = F.softmax(teacher_logits / T, dim=-1)

        kd_loss = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (T * T)

        loss = self.alpha * ce_loss + (1.0 - self.alpha) * kd_loss
        return (loss, student_outputs) if return_outputs else loss



student_args = TrainingArguments(
    output_dir="student-distilbert-kd-restaurants",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=200,
    seed=SEED,
    report_to="none",
    save_strategy="no",
)

kd_trainer = KDTrainer(
    teacher_model=teacher_trainer.model,
    temperature=TEMPERATURE,
    alpha=ALPHA,
    model=student,
    args=student_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

kd_trainer.train()
print("Student KD eval:", kd_trainer.evaluate())



Casting the dataset:   0%|          | 0/8762 [00:00<?, ? examples/s]

Map:   0%|          | 0/7009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1753 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\cengi\AppData\Local\Temp\ipykernel_22784\1248232504.py:143: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  teacher_trainer = Trainer(
c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
200,0.212500
400,0.147400
600,0.098900
800,0.092000
1000,0.067400
1200,0.052700


Teacher eval: {'eval_loss': 0.1586766093969345, 'eval_accuracy': 0.9652025099828865, 'eval_runtime': 105.5635, 'eval_samples_per_second': 16.606, 'eval_steps_per_second': 0.521, 'epoch': 3.0}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\cengi\AppData\Local\Temp\ipykernel_22784\1248232504.py:175: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `KDTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
c:\Users\cengi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
200,0.455600
400,0.205900
600,0.144900
800,0.112300
1000,0.080700
1200,0.057400


Student KD eval: {'eval_loss': 0.12040556222200394, 'eval_accuracy': 0.9646320593268682, 'eval_runtime': 163.8888, 'eval_samples_per_second': 10.696, 'eval_steps_per_second': 0.336, 'epoch': 3.0}
